<a href="https://colab.research.google.com/github/jay2jaykp/ML/blob/master/Online%20Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
#from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
from collections import defaultdict
from functools import partial
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure 
import time
import argparse

In [3]:
#upload data
data = pd.read_csv('cheetah.cs.fiu.edu-110108-113008.1.blkparse', header=None)

#split columns
df = pd.DataFrame(data[0].str.split(' ',9).tolist())

#extract one column --> blocktrace sequence
blocktrace = df[3].tolist()
#convert element to int from str
blocktrace = np.array([int(x) for x in blocktrace])

In [5]:
df.head()

,0,1,2,3,4,5,6,7,8
0,89966811921625,4257,nfsd,488846200,8,W,6,0,2c4787c7671c2f638777c4ab72859ce1
1,89967003980393,4253,nfsd,508455704,8,W,6,0,88b93b628d84082186026d9da044f173
2,89967004049935,4253,nfsd,508455712,8,W,6,0,b5e9f4e5ab62a4fff5313a606b0ad4e3
3,89967004093615,4253,nfsd,508455720,8,W,6,0,7af5556ad4d45263bcd8358867333dd4
4,89967324150054,4254,nfsd,508500288,8,W,6,0,88b93b628d84082186026d9da044f173


In [40]:
blocktrace = df[[0,3]]
blocktrace.columns = ['timestamp', 'block']
blocktrace = blocktrace.apply(pd.to_numeric)
blocktrace.head()

,timestamp,block
0,89966811921625,488846200
1,89967003980393,508455704
2,89967004049935,508455712
3,89967004093615,508455720
4,89967324150054,508500288


In [49]:
unique_block = blocktrace['block'].unique()
unique_block

array([488846200, 508455704, 508455712, ...,  74832032,  74836592,
          748320], dtype=int64)

In [ ]:
freq = {}
for i in (unique_block):
    t =blocktrace.loc[blocktrace['block'] == i ]
    t['freq'] = (t['timestamp'].shift(-1) - t['timestamp']) / 1000000000
    total = t['freq'].sum() / t.shape[0]
    freq[i] = total

In [44]:
t =blocktrace.loc[blocktrace['block'] == 488846200 ]
t['freq'] = (t['timestamp'].shift(-1) - t['timestamp']) / 1000000000
t.head()

C:\Users\Jay\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,timestamp,block,freq
0,89966811921625,488846200,3.203743
3007,89970015664252,488846200,0.447266
3920,89970462930696,488846200,7.024705
7119,89977487635675,488846200,0.404613
7121,89977892249173,488846200,18.042932


In [47]:
total = t['freq'].sum() / t.shape[0]
total

31.08903011429535

In [46]:
t.shape[0]

237

In [20]:
t[0] = t[0].apply(pd.to_numeric)

C:\Users\Jay\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [28]:
t['freq'] = t[0].shift(-1) - t[0]

C:\Users\Jay\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
t.head()

,0,1,2,3,4,5,6,7,8,freq
0,89966811921625,4257,nfsd,488846200,8,W,6,0,2c4787c7671c2f638777c4ab72859ce1,7.770000e+02
3007,89970015664252,4254,nfsd,488846200,8,W,6,0,56dfdc25ec136fbce64f7fe7e336b9ff,4.472664e+08
3920,89970462930696,4253,nfsd,488846200,8,W,6,0,684fb98a8bd827a7ff80a930255fe6da,7.024705e+09
7119,89977487635675,4250,nfsd,488846200,8,W,6,0,56dfdc25ec136fbce64f7fe7e336b9ff,4.046135e+08
7121,89977892249173,4250,nfsd,488846200,8,W,6,0,684fb98a8bd827a7ff80a930255fe6da,1.804293e+10


In [23]:
from datetime import datetime

In [31]:
df[3].unique().shape

(671728,)

In [33]:
t['freq'][0] = 777

C:\Users\Jay\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Jay\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2961: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
